# Extract and build dataset about Nicaragua's Municipal Elections Results 2022

In [362]:
pd.reset_option('all')

/Users/leona/opt/anaconda3/lib/python3.9/site-packages/pandas/_config/config.py:653: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
/Users/leona/opt/anaconda3/lib/python3.9/site-packages/pandas/_config/config.py:653: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [2]:
# Import libraries

import camelot as cam
import pandas as pd
import glob
from fuzzywuzzy import process,fuzz

In [3]:
# Read pdf using CAMELOT

vot_mun_2022 = cam.read_pdf('/Users/leona/Documents/Trabajos/UA/gaceta 14 noviembre 2022.pdf',pages='17-27')

In [4]:
# TO KNOW NUMBER OF TABLES

vot_mun_2022.n

37

In [322]:
# TO ADD ALL THE TABLES IN A SINGLE DATAFRAME

v = []
for i,table in enumerate(vot_mun_2022):
    v.append(table.df)
w = pd.concat(v)

w

,0,1,2,3,4,5,6,7,8
0,MUNICIPIO,ALCALDE Y VICEALCALDE,,,,,,,
1,,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,"19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,"13,542","13,016","3,295","8,098",NP,583,524,516
...,...,...,...,...,...,...,...,...,...
8,POTOSI,"6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,"19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,"4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,"8,301","7,961",426,"7,120",NP,220,195,NP


### The dataframe extracted is a bit messy, so we have to Rename all columns with the ones we need

In [323]:
# RENAME ALL COLUMNS


w.rename(columns={0:'Municipios', 1:'Votos depositados', 2:'Votos Validos', 3:'PLC', 4:'AFSLN', 
                  5:'YATAMA', 6:'ALN', 7:'APRE', 8:'APLI'}, inplace=True)
w

,Municipios,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
0,MUNICIPIO,ALCALDE Y VICEALCALDE,,,,,,,
1,,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,"19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,"13,542","13,016","3,295","8,098",NP,583,524,516
...,...,...,...,...,...,...,...,...,...
8,POTOSI,"6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,"19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,"4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,"8,301","7,961",426,"7,120",NP,220,195,NP


### Before going to the cleaning part, It is necessary to create a column to know if its an election for "Alcalde y vicealcalde" or "Concejales municipales". So, it is necessary to create a copy of 'Votos depositados' columns.

In [324]:
w.insert(loc=1, column='Eleccion1',  value=w['Votos depositados'])
w

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
0,MUNICIPIO,ALCALDE Y VICEALCALDE,ALCALDE Y VICEALCALDE,,,,,,,
1,,VOTOS \nDEPOSITADOS,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,,,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,"19,911","19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,"13,542","13,542","13,016","3,295","8,098",NP,583,524,516
...,...,...,...,...,...,...,...,...,...,...
8,POTOSI,"6,592","6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,"19,759","19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,"4,570","4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,"8,301","8,301","7,961",426,"7,120",NP,220,195,NP


## CLEANING THE DATAFRAME

### From the column created, it will be  specified "Alcalde y vicealcalde" or "Concejales municipales"
### First, delete numerical values

In [325]:
w['Eleccion1'] = w['Eleccion1'].str.replace('\d+', '')

/var/folders/dx/gq6f8_595tsd02m__nff4q480000gn/T/ipykernel_55931/2737749466.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  w['Eleccion1'] = w['Eleccion1'].str.replace('\d+', '')


In [326]:
w

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
0,MUNICIPIO,ALCALDE Y VICEALCALDE,ALCALDE Y VICEALCALDE,,,,,,,
1,,VOTOS \nDEPOSITADOS,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,,,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,",","19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,",","13,542","13,016","3,295","8,098",NP,583,524,516
...,...,...,...,...,...,...,...,...,...,...
8,POTOSI,",","6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,",","19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,",","4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,",","8,301","7,961",426,"7,120",NP,220,195,NP


In [327]:
# CHECK THE UNIQUE VALUES of 'Eleccion1' column

w['Eleccion1'].unique()

array(['ALCALDE Y VICEALCALDE', 'VOTOS \nDEPOSITADOS', '', ',',
       'CONCEJALES MUNICIPALES', '.', ',\n¡', ' VOTOS \nDEPOSITADOS'],
      dtype=object)

In [328]:
# REPLACING 'VOTOS \nDEPOSITADOS'

w['Eleccion1'] = w['Eleccion1'].str.replace('VOTOS \nDEPOSITADOS', '')
w['Eleccion1'].value_counts()

,                         300
                           69
ALCALDE Y VICEALCALDE      17
CONCEJALES MUNICIPALES     17
.                           3
,\n¡                        1
                            1
Name: Eleccion1, dtype: int64

In [329]:
# REPLACING ALL VALUES THAT ARE NOT ALPHABETICS

w['Eleccion1'] = w['Eleccion1'].str.replace(r'\W', " ")
w['Eleccion1'].value_counts()

/var/folders/dx/gq6f8_595tsd02m__nff4q480000gn/T/ipykernel_55931/1406430155.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  w['Eleccion1'] = w['Eleccion1'].str.replace(r'\W', " ")


                          304
                           69
ALCALDE Y VICEALCALDE      17
CONCEJALES MUNICIPALES     17
                            1
Name: Eleccion1, dtype: int64

In [330]:
# REPLACING EMPTY RECORDS FOR NAN VALUES

w['Eleccion1'] = w['Eleccion1'].replace(r'^\s*$', np.nan,regex=True)
w

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
0,MUNICIPIO,ALCALDE Y VICEALCALDE,ALCALDE Y VICEALCALDE,,,,,,,
1,,NaN,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,,NaN,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,NaN,"19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,NaN,"13,542","13,016","3,295","8,098",NP,583,524,516
...,...,...,...,...,...,...,...,...,...,...
8,POTOSI,NaN,"6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,NaN,"19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,NaN,"4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,NaN,"8,301","7,961",426,"7,120",NP,220,195,NP


In [331]:
w['Eleccion1'].value_counts(dropna=False)

NaN                       374
ALCALDE Y VICEALCALDE      17
CONCEJALES MUNICIPALES     17
Name: Eleccion1, dtype: int64

In [332]:
# FLASH FILL TO - FILL NAN WITH THE ABOVE VALUE

w['Eleccion1'] = w['Eleccion1'].ffill()
w

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
0,MUNICIPIO,ALCALDE Y VICEALCALDE,ALCALDE Y VICEALCALDE,,,,,,,
1,,ALCALDE Y VICEALCALDE,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,,ALCALDE Y VICEALCALDE,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,ALCALDE Y VICEALCALDE,"19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,ALCALDE Y VICEALCALDE,"13,542","13,016","3,295","8,098",NP,583,524,516
...,...,...,...,...,...,...,...,...,...,...
8,POTOSI,CONCEJALES MUNICIPALES,"6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,CONCEJALES MUNICIPALES,"19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,CONCEJALES MUNICIPALES,"4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,CONCEJALES MUNICIPALES,"8,301","7,961",426,"7,120",NP,220,195,NP


### 2. CLEANING 'MUNICIPIOS' COLUMN

In [333]:
w['Municipios'].sort_values().unique()

array(['', 'ACHUAPA', 'ACOYA PA', 'ACOYAPA', 'ALTAGRACIA', 'BELEN',
       'BLUEFIELDS', 'BOACO', 'BOCANA DE PAIWAS', 'BONANZA',
       'BUENOS AIRES', 'CAMOAPA', 'CARDENAS', 'CATARINA', 'CHICIIIGALPA',
       'CHINANDEGA', 'CIIIC1I1GALPA', 'CINCO PINOS', 'CIUDAD ANTIGUA',
       'CIUDAD DARIO', 'CIUDAD SANDINO', 'COMALAPA', 'CONDEGA', 'CORINTO',
       'CORN ISLAND', 'DIPILTO', 'DIRIA', 'DIRIAMBA', 'DIRIOMO',
       'DOLORES', 'EL ALMENDRO', 'EL AYOTE', 'EL CASTILLO', 'EL CORAL',
       'EL CRUCERO', 'EL JICARAL', 'EL JICARO', 'EL RAMA', 'EL REALEJO',
       'EL ROSARIO', 'EL SAUCE', 'EL TUMA LA DALIA', 'EL VIEJO', 'ELCUA',
       'ELTORTUGUERO', 'ELTUMA LA DALIA', 'ESQUIPULAS', 'ESTELI',
       'GRANADA', 'JALAPA', 'JINOTEGA', 'JINOTEPE', 'JUIGALPA',
       'KUKRA HILL', 'KUKRA IIILL', 'LA CONCEPCION', 'LA CONCORDIA',
       'LA CONQUISTA', 'LA CRUZ DE RIO GRANDE',
       'LA DESEMBOCADURA DE \nRIO GRANDE', 'LA LIBERTAD', 'LA PAZ CENTRO',
       'LA PAZ DE CARAZO', 'LA TRINIDAD', 'LA

### The 'Municipios' column is messy, because the names are not correct.

In [334]:
# First, delete white spaces at the beginning and at the end.

w['Municipios'] = w['Municipios'].str.strip()
w['Municipios'].unique()

array(['MUNICIPIO', '', 'BOACO', 'CAMOAPA', 'SAN JOSE DE LOS REMATES',
       'SAN LORENZO', 'SANTA LUCIA', 'TEUSTEPE', 'DIRIAMBA', 'DOLORES',
       'EL ROSARIO', 'JINOTEPE', 'LA CONQUISTA', 'LA PAZ DE CARAZO',
       'SAN MARCOS', 'SANTA TERESA', 'CIIIC1I1GALPA', 'CHINANDEGA',
       'CINCO PINOS', 'CORINTO', 'EL REALEJO', 'EL VIEJO', 'POSOLTEGA',
       'PUERTO MORAZAN', 'SAN FRANCISCO DEL \nNORTE',
       'SAN PEDRO DEL NORTE', 'SANTO TOMAS DEL NORTE', 'SOMOTILLO',
       'VILLANUEVA', 'CHICIIIGALPA', 'SANTO TOMAS DEL ÑOR TE', 'ACOYA PA',
       'COMALAPA', 'EL AYOTE', 'EL CORAL', 'EL RAMA', 'JUIGALPA',
       'LA LIBERTAD', 'MUELLE DE LOS BUEYES', 'NUEVA GUINEA',
       'SAN FRANCISCO DE CUAPA', 'SAN PEDRO DE LOVAGO', 'SANTO DOMINGO',
       'SANTO TOMAS', 'VILLA SANDINO', 'ACOYAPA', 'CONDEGA', 'ESTELI',
       'LA TRINIDAD', 'PUEBLO NUEVO', 'SAN JUAN DE LIMAY', 'SAN NICOLAS',
       'DIRIA', 'DIRIOMO', 'GRANADA', 'NANDAIME', 'ELCUA', 'JINOTEGA',
       'LA CONCORDIA', 'SAN JOSE D

In [335]:
w['Municipios'] = w['Municipios'].replace('  ','', regex=True)
w['Municipios'].sort_values().unique()

array(['', 'ACHUAPA', 'ACOYA PA', 'ACOYAPA', 'ALTAGRACIA', 'BELEN',
       'BLUEFIELDS', 'BOACO', 'BOCANA DE PAIWAS', 'BONANZA',
       'BUENOS AIRES', 'CAMOAPA', 'CARDENAS', 'CATARINA', 'CHICIIIGALPA',
       'CHINANDEGA', 'CIIIC1I1GALPA', 'CINCO PINOS', 'CIUDAD ANTIGUA',
       'CIUDAD DARIO', 'CIUDAD SANDINO', 'COMALAPA', 'CONDEGA', 'CORINTO',
       'CORN ISLAND', 'DIPILTO', 'DIRIA', 'DIRIAMBA', 'DIRIOMO',
       'DOLORES', 'EL ALMENDRO', 'EL AYOTE', 'EL CASTILLO', 'EL CORAL',
       'EL CRUCERO', 'EL JICARAL', 'EL JICARO', 'EL RAMA', 'EL REALEJO',
       'EL ROSARIO', 'EL SAUCE', 'EL TUMA LA DALIA', 'EL VIEJO', 'ELCUA',
       'ELTORTUGUERO', 'ELTUMA LA DALIA', 'ESQUIPULAS', 'ESTELI',
       'GRANADA', 'JALAPA', 'JINOTEGA', 'JINOTEPE', 'JUIGALPA',
       'KUKRA HILL', 'KUKRA IIILL', 'LA CONCEPCION', 'LA CONCORDIA',
       'LA CONQUISTA', 'LA CRUZ DE RIO GRANDE',
       'LA DESEMBOCADURA DE \nRIO GRANDE', 'LA LIBERTAD', 'LA PAZ CENTRO',
       'LA PAZ DE CARAZO', 'LA TRINIDAD', 'LA

In [336]:
w[w['Municipios']=='S1UNA']

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
8,S1UNA,ALCALDE Y VICEALCALDE,"25,023\n¡","23,977","3,886","16,594",66,"1,151","1,138","1,142"


In [337]:
# SINCE THE COLUMN HAS EMPTY VALUES, WE ARE GOING TO REPLACE THEM BY NAN

w['Municipios'] = w['Municipios'].replace(r'^\s*$', np.nan,regex=True)
w['Municipios'].sort_values().unique()

array(['ACHUAPA', 'ACOYA PA', 'ACOYAPA', 'ALTAGRACIA', 'BELEN',
       'BLUEFIELDS', 'BOACO', 'BOCANA DE PAIWAS', 'BONANZA',
       'BUENOS AIRES', 'CAMOAPA', 'CARDENAS', 'CATARINA', 'CHICIIIGALPA',
       'CHINANDEGA', 'CIIIC1I1GALPA', 'CINCO PINOS', 'CIUDAD ANTIGUA',
       'CIUDAD DARIO', 'CIUDAD SANDINO', 'COMALAPA', 'CONDEGA', 'CORINTO',
       'CORN ISLAND', 'DIPILTO', 'DIRIA', 'DIRIAMBA', 'DIRIOMO',
       'DOLORES', 'EL ALMENDRO', 'EL AYOTE', 'EL CASTILLO', 'EL CORAL',
       'EL CRUCERO', 'EL JICARAL', 'EL JICARO', 'EL RAMA', 'EL REALEJO',
       'EL ROSARIO', 'EL SAUCE', 'EL TUMA LA DALIA', 'EL VIEJO', 'ELCUA',
       'ELTORTUGUERO', 'ELTUMA LA DALIA', 'ESQUIPULAS', 'ESTELI',
       'GRANADA', 'JALAPA', 'JINOTEGA', 'JINOTEPE', 'JUIGALPA',
       'KUKRA HILL', 'KUKRA IIILL', 'LA CONCEPCION', 'LA CONCORDIA',
       'LA CONQUISTA', 'LA CRUZ DE RIO GRANDE',
       'LA DESEMBOCADURA DE \nRIO GRANDE', 'LA LIBERTAD', 'LA PAZ CENTRO',
       'LA PAZ DE CARAZO', 'LA TRINIDAD', 'LAGUNA

In [338]:
w

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
0,MUNICIPIO,ALCALDE Y VICEALCALDE,ALCALDE Y VICEALCALDE,,,,,,,
1,NaN,ALCALDE Y VICEALCALDE,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,NaN,ALCALDE Y VICEALCALDE,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,ALCALDE Y VICEALCALDE,"19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,ALCALDE Y VICEALCALDE,"13,542","13,016","3,295","8,098",NP,583,524,516
...,...,...,...,...,...,...,...,...,...,...
8,POTOSI,CONCEJALES MUNICIPALES,"6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,CONCEJALES MUNICIPALES,"19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,CONCEJALES MUNICIPALES,"4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,CONCEJALES MUNICIPALES,"8,301","7,961",426,"7,120",NP,220,195,NP


In [339]:
# DROP ROWS WITH THE RECORD == 'MUNICIPIO'

w=w[~w['Municipios'].isin(['MUNICIPIO'])]
w

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
1,NaN,ALCALDE Y VICEALCALDE,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,NaN,ALCALDE Y VICEALCALDE,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,ALCALDE Y VICEALCALDE,"19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,ALCALDE Y VICEALCALDE,"13,542","13,016","3,295","8,098",NP,583,524,516
5,SAN JOSE DE LOS REMATES,ALCALDE Y VICEALCALDE,"4,300","4,080","1,613","2,030",NP,149,132,156
...,...,...,...,...,...,...,...,...,...,...
8,POTOSI,CONCEJALES MUNICIPALES,"6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,CONCEJALES MUNICIPALES,"19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,CONCEJALES MUNICIPALES,"4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,CONCEJALES MUNICIPALES,"8,301","7,961",426,"7,120",NP,220,195,NP


In [340]:
# CHECK THE NAN VALUES

w[w.Municipios.isnull()]

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
1,NaN,ALCALDE Y VICEALCALDE,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,NaN,ALCALDE Y VICEALCALDE,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
1,NaN,CONCEJALES MUNICIPALES,VOTOS \nDEPOSITADOS,VOTOS\nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,NaN,CONCEJALES MUNICIPALES,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
1,NaN,ALCALDE Y VICEALCALDE,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
...,...,...,...,...,...,...,...,...,...,...
2,NaN,CONCEJALES MUNICIPALES,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
1,NaN,ALCALDE Y VICEALCALDE,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,
2,NaN,ALCALDE Y VICEALCALDE,,,PLC,AFSLN,YATAMA,ALN,APRE,APLI
1,NaN,CONCEJALES MUNICIPALES,VOTOS \nDEPOSITADOS,VOTOS \nVÁLIDOS,VOTOS VÁLIDOS POR PARTIDO,,,,,


In [341]:
# DROP NAN VALUES

w = w.dropna(subset=['Municipios'])
w

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI
3,BOACO,ALCALDE Y VICEALCALDE,"19,911","19,020","1,704","14,810",NP,872,832,802
4,CAMOAPA,ALCALDE Y VICEALCALDE,"13,542","13,016","3,295","8,098",NP,583,524,516
5,SAN JOSE DE LOS REMATES,ALCALDE Y VICEALCALDE,"4,300","4,080","1,613","2,030",NP,149,132,156
6,SAN LORENZO,ALCALDE Y VICEALCALDE,"9,005","8,856",745,"7,223",NP,272,312,304
7,SANTA LUCIA,ALCALDE Y VICEALCALDE,"2,597","2,537",382,"2,021",NP,20,97,17
...,...,...,...,...,...,...,...,...,...,...
8,POTOSI,CONCEJALES MUNICIPALES,"6,592","6,349",303,"5,732",NP,199,NP,115
9,RIVAS,CONCEJALES MUNICIPALES,"19,759","18,861",967,"16,659",NP,818,NP,417
10,SAN JORGE,CONCEJALES MUNICIPALES,"4,570","4,353",350,"3,782",NP,132,NP,89
11,SAN JUAN DEL SUR,CONCEJALES MUNICIPALES,"8,301","7,961",426,"7,120",NP,220,195,NP


In [342]:
w.Municipios.sort_values().unique()

array(['ACHUAPA', 'ACOYA PA', 'ACOYAPA', 'ALTAGRACIA', 'BELEN',
       'BLUEFIELDS', 'BOACO', 'BOCANA DE PAIWAS', 'BONANZA',
       'BUENOS AIRES', 'CAMOAPA', 'CARDENAS', 'CATARINA', 'CHICIIIGALPA',
       'CHINANDEGA', 'CIIIC1I1GALPA', 'CINCO PINOS', 'CIUDAD ANTIGUA',
       'CIUDAD DARIO', 'CIUDAD SANDINO', 'COMALAPA', 'CONDEGA', 'CORINTO',
       'CORN ISLAND', 'DIPILTO', 'DIRIA', 'DIRIAMBA', 'DIRIOMO',
       'DOLORES', 'EL ALMENDRO', 'EL AYOTE', 'EL CASTILLO', 'EL CORAL',
       'EL CRUCERO', 'EL JICARAL', 'EL JICARO', 'EL RAMA', 'EL REALEJO',
       'EL ROSARIO', 'EL SAUCE', 'EL TUMA LA DALIA', 'EL VIEJO', 'ELCUA',
       'ELTORTUGUERO', 'ELTUMA LA DALIA', 'ESQUIPULAS', 'ESTELI',
       'GRANADA', 'JALAPA', 'JINOTEGA', 'JINOTEPE', 'JUIGALPA',
       'KUKRA HILL', 'KUKRA IIILL', 'LA CONCEPCION', 'LA CONCORDIA',
       'LA CONQUISTA', 'LA CRUZ DE RIO GRANDE',
       'LA DESEMBOCADURA DE \nRIO GRANDE', 'LA LIBERTAD', 'LA PAZ CENTRO',
       'LA PAZ DE CARAZO', 'LA TRINIDAD', 'LAGUNA

### Now, i'm going to use FuzzyWuzzy to get the real names of every municipio. In this case, i will create two lists with the unique values in 'Municipios' column and the correct list o 'Municipios'

In [343]:
munic_corr = ['Achuapa', 'Acoyapa', 'Altagracia', 'Belén', 'Bluefields', 'Boaco', 
              'Bonanza', 'Buenos Aires', 'Camoapa', 'Cárdenas', 'Catarina', 'Chichigalpa', 'Chinandega',
              'Cinco Pinos', 'Ciudad Antigua', 'Ciudad Darío', 'Ciudad Sandino', 'Comalapa',
 'Condega', 'Corinto', 'Corn Island', 'Desembocadura de Río Grande', 'Dipilto',
 'Diriá', 'Diriamba', 'Diriomo', 'Dolores', 'El Almendro', 'El Ayote', 'El Castillo',
 'El Coral', 'El Crucero', 'El Cuá', 'El Jicaral', 'El Rama', 'El Realejo',
 'El Rosario', 'El Sauce', 'El Tortuguero', 'El Tuma - La Dalia',
 'El Viejo', 'Esquipulas', 'Estelí', 'Granada', 'Jalapa', 'Jícaro',
 'Jinotega', 'Jinotepe', 'Juigalpa', 'Kukrahill', 'La Concepción', 'La Concordia', 'La Conquista',
 'La Cruz de Río Grande', 'La Libertad', 'La Paz Centro', 'La Paz de Carazo', 'La Trinidad',
 'Laguna de Perlas', 'Larreynaga', 'Las Sabanas', 'León', 'Macuelizo', 'Managua', 'Masatepe',
 'Masaya', 'Matagalpa', 'Mateare', 'Matiguás', 'Morrito', 'Moyogalpa', 'Mozonte', 'Muelle de Los Bueyes',
 'Mulukukú', 'Murra', 'Muy Muy', 'Nagarote', 'Nandaime', 'Nandasmo', 'Nindirí', 'Niquinohomo',
 'Nueva Guinea', 'Ocotal', 'Paiwas', 'Palacagüina', 'Posoltega', 'Potosí', 'Prinzapolka',
 'Pueblo Nuevo', 'Puerto Cabezas', 'Puerto Morazán', 'Quezalguaque', 'Quilalí', 'Rancho Grande', 
              'Río Blanco', 'Rivas', 'Rosita', 'San Carlos', 'San Dionisio', 'San Fernando',
              'San Francisco de Cuapa', 'San Francisco del Norte', 'San Francisco Libre', 'San Isidro',
 'San Jorge', 'San José de Bocay', 'San José de Cusmapa', 'San José de Los Remates', 'San Juan de Limay',
 'San Juan de Nicaragua', 'San Juan de Oriente', 'San Juan del Río Coco', 'San Juan del Sur',
 'San Lorenzo', 'San Lucas', 'San Marcos', 'San Miguelito', 'San Nicolás', 'San Pedro de Lóvago',
 'San Pedro del Norte', 'San Rafael del Norte', 'San Rafael del Sur', 'San Ramón', 'San Sebastián de Yalí',
 'Santa Lucía', 'Santa María', 'Santa María de Pantasma', 'Santa Rosa del Peñón', 'Santa Teresa',
 'Santo Domingo', 'Santo Tomás', 'Santo Tomás del Norte', 'Sébaco', 'Siuna', 'Somotillo', 'Somoto',
 'Telica', 'Telpaneca', 'Terrabona', 'Teustepe', 'Ticuantepe', 'Tipitapa', 'Tisma', 'Tola',
 'Totogalpa', 'Villa El Carmen', 'Villa Sandino', 'Villanueva', 'Waslala', 'Waspam', 'Wiwilí de Jinotega',
 'Wiwilí de Nueva Segovia', 'Yalagüina']

In [344]:
list1=w.Municipios.tolist()

In [345]:
mat1 = []
mat2 = []
p=[]
for i in list1:
    mat1.append(process.extract(i, munic_corr,limit=1, scorer=fuzz.token_set_ratio))
w['matches'] = mat1

/var/folders/dx/gq6f8_595tsd02m__nff4q480000gn/T/ipykernel_55931/2207590803.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w['matches'] = mat1


In [346]:
w.sort_values(by='Municipios')

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,matches
3,ACHUAPA,CONCEJALES MUNICIPALES,"4,050","3,952",477,"3,213",NP,95,94,73,"[(Achuapa, 100)]"
3,ACHUAPA,ALCALDE Y VICEALCALDE,"4,050","3,953",576,"2,980",NP,124,132,141,"[(Achuapa, 100)]"
3,ACOYA PA,ALCALDE Y VICEALCALDE,"6,889","6,799",613,"5,558",NP,209,214,205,"[(Acoyapa, 93)]"
3,ACOYAPA,CONCEJALES MUNICIPALES,"6,889","6,800",478,"5,906",NP,162,153,101,"[(Acoyapa, 100)]"
3,ALTAGRACIA,ALCALDE Y VICEALCALDE,"8,130","7,811",766,"6,300",NP,NP,370,375,"[(Altagracia, 100)]"
...,...,...,...,...,...,...,...,...,...,...,...
10,WIWILI-JINOTEGA,ALCALDE Y VICEALCALDE,"19,347","19,049","1,403","16,258",NP,474,453,461,"[(Jinotega, 100)]"
14,WIWILI-NUEVA SEGOVIA,ALCALDE Y VICEALCALDE,"5,860","5,776",915,"3,952",NP,288,294,327,"[(Wiwilí de Nueva Segovia, 90)]"
3,WIWILLNUEVA SEGOVIA,CONCEJALES MUNICIPALES,"5,860","5,777",648,4.557,NP,217,203,152,"[(Wiwilí de Nueva Segovia, 63)]"
11,YALAGUINA,CONCEJALES MUNICIPALES,"3,975","3,898",149,"3,426",NP,144,102,77,"[(Yalagüina, 94)]"


In [347]:
threshold=50

for j in w['matches']:
    for k in j:
        
        if k[1] >= threshold:
            p.append(k[0])
              
    mat2.append(",".join(p))
    p = []
      
# storing the resultant matches 
# back to dframe1
w['matches'] = mat2
  
w.sort_values(by='Municipios')

/var/folders/dx/gq6f8_595tsd02m__nff4q480000gn/T/ipykernel_55931/1559205059.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w['matches'] = mat2


,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,matches
3,ACHUAPA,CONCEJALES MUNICIPALES,"4,050","3,952",477,"3,213",NP,95,94,73,Achuapa
3,ACHUAPA,ALCALDE Y VICEALCALDE,"4,050","3,953",576,"2,980",NP,124,132,141,Achuapa
3,ACOYA PA,ALCALDE Y VICEALCALDE,"6,889","6,799",613,"5,558",NP,209,214,205,Acoyapa
3,ACOYAPA,CONCEJALES MUNICIPALES,"6,889","6,800",478,"5,906",NP,162,153,101,Acoyapa
3,ALTAGRACIA,ALCALDE Y VICEALCALDE,"8,130","7,811",766,"6,300",NP,NP,370,375,Altagracia
...,...,...,...,...,...,...,...,...,...,...,...
10,WIWILI-JINOTEGA,ALCALDE Y VICEALCALDE,"19,347","19,049","1,403","16,258",NP,474,453,461,Jinotega
14,WIWILI-NUEVA SEGOVIA,ALCALDE Y VICEALCALDE,"5,860","5,776",915,"3,952",NP,288,294,327,Wiwilí de Nueva Segovia
3,WIWILLNUEVA SEGOVIA,CONCEJALES MUNICIPALES,"5,860","5,777",648,4.557,NP,217,203,152,Wiwilí de Nueva Segovia
11,YALAGUINA,CONCEJALES MUNICIPALES,"3,975","3,898",149,"3,426",NP,144,102,77,Yalagüina


In [348]:
w.rename({'matches':'Municipio'}, axis=1, inplace=True)
w.head()

/var/folders/dx/gq6f8_595tsd02m__nff4q480000gn/T/ipykernel_55931/3323490231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w.rename({'matches':'Municipio'}, axis=1, inplace=True)


,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio
3,BOACO,ALCALDE Y VICEALCALDE,"19,911","19,020","1,704","14,810",NP,872,832,802,Boaco
4,CAMOAPA,ALCALDE Y VICEALCALDE,"13,542","13,016","3,295","8,098",NP,583,524,516,Camoapa
5,SAN JOSE DE LOS REMATES,ALCALDE Y VICEALCALDE,"4,300","4,080","1,613","2,030",NP,149,132,156,San José de Los Remates
6,SAN LORENZO,ALCALDE Y VICEALCALDE,"9,005","8,856",745,"7,223",NP,272,312,304,San Lorenzo
7,SANTA LUCIA,ALCALDE Y VICEALCALDE,"2,597","2,537",382,"2,021",NP,20,97,17,Santa Lucía


In [349]:
w1=w.reset_index(drop=True).copy()
w1

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio
0,BOACO,ALCALDE Y VICEALCALDE,"19,911","19,020","1,704","14,810",NP,872,832,802,Boaco
1,CAMOAPA,ALCALDE Y VICEALCALDE,"13,542","13,016","3,295","8,098",NP,583,524,516,Camoapa
2,SAN JOSE DE LOS REMATES,ALCALDE Y VICEALCALDE,"4,300","4,080","1,613","2,030",NP,149,132,156,San José de Los Remates
3,SAN LORENZO,ALCALDE Y VICEALCALDE,"9,005","8,856",745,"7,223",NP,272,312,304,San Lorenzo
4,SANTA LUCIA,ALCALDE Y VICEALCALDE,"2,597","2,537",382,"2,021",NP,20,97,17,Santa Lucía
...,...,...,...,...,...,...,...,...,...,...,...
301,POTOSI,CONCEJALES MUNICIPALES,"6,592","6,349",303,"5,732",NP,199,NP,115,Potosí
302,RIVAS,CONCEJALES MUNICIPALES,"19,759","18,861",967,"16,659",NP,818,NP,417,Rivas
303,SAN JORGE,CONCEJALES MUNICIPALES,"4,570","4,353",350,"3,782",NP,132,NP,89,San Jorge
304,SAN JUAN DEL SUR,CONCEJALES MUNICIPALES,"8,301","7,961",426,"7,120",NP,220,195,NP,San Juan del Sur


In [350]:
w1.Municipio.value_counts()

El Jicaral           4
Jinotega             4
Telica               4
Boaco                2
San Dionisio         2
                    ..
Granada              2
Nandaime             2
La Concordia         2
San José de Bocay    2
Tola                 2
Name: Municipio, Length: 150, dtype: int64

### A final cleaning step: i found that at least 3 records were not changed correctly, so i will change it manually.

In [351]:
w1[w1['Municipio'].str.contains('El Jicaral|Jinotega|Telica')]

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio
102,ELCUA,ALCALDE Y VICEALCALDE,"15,901","15,172","2,494","11,317",NP,694,667,NP,Telica
103,JINOTEGA,ALCALDE Y VICEALCALDE,"39,684","37,451","3,752","28,411",NP,"1,941","1,649","1,698",Jinotega
109,WIWILI-JINOTEGA,ALCALDE Y VICEALCALDE,"19,347","19,049","1,403","16,258",NP,474,453,461,Jinotega
110,ELCUA,CONCEJALES MUNICIPALES,"15,885","15,147","1,824","12,321",NP,533,469,NP,Telica
111,JINOTEGA,CONCEJALES MUNICIPALES,"40,010","37,739","2,774","31,382",NP,"1,499","1,175",909,Jinotega
117,W1WILI-JINOTEGA,CONCEJALES MUNICIPALES,"19,333","19,040","1,045","17,070",NP,363,324,238,Jinotega
119,EL JICARAL,ALCALDE Y VICEALCALDE,"3,866","3,766",327,"3,024",NP,145,140,130,El Jicaral
127,TEL1CA,ALCALDE Y VICEALCALDE,"10,034","9,747",604,"8,455",NP,347,341,NP,Telica
129,EL JICARAL,CONCEJALES MUNICIPALES,"3,866","3,766",233,"3,256",NP,114,101,62,El Jicaral
137,TELICA,CONCEJALES MUNICIPALES,"10,034","9,747",459,"8,777",NP,267,244,NP,Telica


In [352]:
w1.loc[[109,117],'Municipio'] = 'Wiwilí de Jinotega'
w1[w1['Municipio'].str.contains('El Jicaral|Jinotega|Telica')]

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio
102,ELCUA,ALCALDE Y VICEALCALDE,"15,901","15,172","2,494","11,317",NP,694,667,NP,Telica
103,JINOTEGA,ALCALDE Y VICEALCALDE,"39,684","37,451","3,752","28,411",NP,"1,941","1,649","1,698",Jinotega
109,WIWILI-JINOTEGA,ALCALDE Y VICEALCALDE,"19,347","19,049","1,403","16,258",NP,474,453,461,Wiwilí de Jinotega
110,ELCUA,CONCEJALES MUNICIPALES,"15,885","15,147","1,824","12,321",NP,533,469,NP,Telica
111,JINOTEGA,CONCEJALES MUNICIPALES,"40,010","37,739","2,774","31,382",NP,"1,499","1,175",909,Jinotega
117,W1WILI-JINOTEGA,CONCEJALES MUNICIPALES,"19,333","19,040","1,045","17,070",NP,363,324,238,Wiwilí de Jinotega
119,EL JICARAL,ALCALDE Y VICEALCALDE,"3,866","3,766",327,"3,024",NP,145,140,130,El Jicaral
127,TEL1CA,ALCALDE Y VICEALCALDE,"10,034","9,747",604,"8,455",NP,347,341,NP,Telica
129,EL JICARAL,CONCEJALES MUNICIPALES,"3,866","3,766",233,"3,256",NP,114,101,62,El Jicaral
137,TELICA,CONCEJALES MUNICIPALES,"10,034","9,747",459,"8,777",NP,267,244,NP,Telica


In [353]:
w1.loc[[102,110],'Municipio'] = 'El Cuá'
w1[w1['Municipio'].str.contains('El Jicaral|Jinotega|Telica')]

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio
103,JINOTEGA,ALCALDE Y VICEALCALDE,"39,684","37,451","3,752","28,411",NP,"1,941","1,649","1,698",Jinotega
109,WIWILI-JINOTEGA,ALCALDE Y VICEALCALDE,"19,347","19,049","1,403","16,258",NP,474,453,461,Wiwilí de Jinotega
111,JINOTEGA,CONCEJALES MUNICIPALES,"40,010","37,739","2,774","31,382",NP,"1,499","1,175",909,Jinotega
117,W1WILI-JINOTEGA,CONCEJALES MUNICIPALES,"19,333","19,040","1,045","17,070",NP,363,324,238,Wiwilí de Jinotega
119,EL JICARAL,ALCALDE Y VICEALCALDE,"3,866","3,766",327,"3,024",NP,145,140,130,El Jicaral
127,TEL1CA,ALCALDE Y VICEALCALDE,"10,034","9,747",604,"8,455",NP,347,341,NP,Telica
129,EL JICARAL,CONCEJALES MUNICIPALES,"3,866","3,766",233,"3,256",NP,114,101,62,El Jicaral
137,TELICA,CONCEJALES MUNICIPALES,"10,034","9,747",459,"8,777",NP,267,244,NP,Telica
222,EL JICARO,ALCALDE Y VICEALCALDE,"8,725","8,574","1,096","6,232",NP,413,405,428,El Jicaral
234,EL JICARO,CONCEJALES MUNICIPALES,"8,725","8,574",784,"6,976",NP,313,287,214,El Jicaral


In [354]:
w1.loc[[222,234],'Municipio'] = 'Jícaro'
w1[w1['Municipio'].str.contains('El Jicaral|Jinotega|Telica')]

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio
103,JINOTEGA,ALCALDE Y VICEALCALDE,"39,684","37,451","3,752","28,411",NP,"1,941","1,649","1,698",Jinotega
109,WIWILI-JINOTEGA,ALCALDE Y VICEALCALDE,"19,347","19,049","1,403","16,258",NP,474,453,461,Wiwilí de Jinotega
111,JINOTEGA,CONCEJALES MUNICIPALES,"40,010","37,739","2,774","31,382",NP,"1,499","1,175",909,Jinotega
117,W1WILI-JINOTEGA,CONCEJALES MUNICIPALES,"19,333","19,040","1,045","17,070",NP,363,324,238,Wiwilí de Jinotega
119,EL JICARAL,ALCALDE Y VICEALCALDE,"3,866","3,766",327,"3,024",NP,145,140,130,El Jicaral
127,TEL1CA,ALCALDE Y VICEALCALDE,"10,034","9,747",604,"8,455",NP,347,341,NP,Telica
129,EL JICARAL,CONCEJALES MUNICIPALES,"3,866","3,766",233,"3,256",NP,114,101,62,El Jicaral
137,TELICA,CONCEJALES MUNICIPALES,"10,034","9,747",459,"8,777",NP,267,244,NP,Telica


In [356]:
pd.set_option('display.max_rows', None)

In [357]:
w1.Municipio.value_counts()

Boaco                          2
San Isidro                     2
Esquipulas                     2
Matagalpa                      2
Matiguás                       2
Muy Muy                        2
Rancho Grande                  2
Río Blanco                     2
San Dionisio                   2
San Ramón                      2
Yalagüina                      2
Sébaco                         2
Terrabona                      2
Waslala                        2
Ciudad Antigua                 2
Dipilto                        2
Jícaro                         2
Jalapa                         2
El Tuma - La Dalia             2
Ciudad Darío                   2
Tisma                          2
San Juan de Oriente            2
El Crucero                     2
Managua                        2
Mateare                        2
San Francisco Libre            2
San Rafael del Sur             2
Ticuantepe                     2
Tipitapa                       2
Villa El Carmen                2
Catarina  

In [360]:
w1.Municipio.sort_values().unique()

array(['Achuapa', 'Acoyapa', 'Altagracia', 'Belén', 'Bluefields', 'Boaco',
       'Bonanza', 'Buenos Aires', 'Camoapa', 'Catarina', 'Chichigalpa',
       'Chinandega', 'Cinco Pinos', 'Ciudad Antigua', 'Ciudad Darío',
       'Ciudad Sandino', 'Comalapa', 'Condega', 'Corinto', 'Corn Island',
       'Cárdenas', 'Desembocadura de Río Grande', 'Dipilto', 'Diriamba',
       'Diriomo', 'Diriá', 'Dolores', 'El Almendro', 'El Ayote',
       'El Castillo', 'El Coral', 'El Crucero', 'El Cuá', 'El Jicaral',
       'El Rama', 'El Realejo', 'El Rosario', 'El Sauce', 'El Tortuguero',
       'El Tuma - La Dalia', 'El Viejo', 'Esquipulas', 'Estelí',
       'Granada', 'Jalapa', 'Jinotega', 'Jinotepe', 'Juigalpa', 'Jícaro',
       'Kukrahill', 'La Concepción', 'La Concordia', 'La Conquista',
       'La Cruz de Río Grande', 'La Libertad', 'La Paz Centro',
       'La Paz de Carazo', 'La Trinidad', 'Laguna de Perlas',
       'Larreynaga', 'Las Sabanas', 'León', 'Macuelizo', 'Managua',
       'Masatepe', 'Ma

### Now, i think is pretty clean and i have the right name of 'Municipio'. Next step is give the right types of data to the numerical ones.

In [361]:
w1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Municipios         306 non-null    object
 1   Eleccion1          306 non-null    object
 2   Votos depositados  306 non-null    object
 3   Votos Validos      306 non-null    object
 4   PLC                306 non-null    object
 5   AFSLN              306 non-null    object
 6   YATAMA             306 non-null    object
 7   ALN                306 non-null    object
 8   APRE               306 non-null    object
 9   APLI               306 non-null    object
 10  Municipio          306 non-null    object
dtypes: object(11)
memory usage: 26.4+ KB


### It can be seen that from column 2 to 9, we have objects type, but these should be numerical as integers.

In [277]:
w1

,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio
3,ALCALDE Y VICEALCALDE,"19,911","19,020","1,704","14,810",NP,872,832,802,Boaco
4,ALCALDE Y VICEALCALDE,"13,542","13,016","3,295","8,098",NP,583,524,516,Camoapa
5,ALCALDE Y VICEALCALDE,"4,300","4,080","1,613","2,030",NP,149,132,156,San José de Los Remates
6,ALCALDE Y VICEALCALDE,"9,005","8,856",745,"7,223",NP,272,312,304,San Lorenzo
7,ALCALDE Y VICEALCALDE,"2,597","2,537",382,"2,021",NP,20,97,17,Santa Lucía
...,...,...,...,...,...,...,...,...,...,...
8,CONCEJALES MUNICIPALES,"6,592","6,349",303,"5,732",NP,199,NP,115,Potosí
9,CONCEJALES MUNICIPALES,"19,759","18,861",967,"16,659",NP,818,NP,417,Rivas
10,CONCEJALES MUNICIPALES,"4,570","4,353",350,"3,782",NP,132,NP,89,San Jorge
11,CONCEJALES MUNICIPALES,"8,301","7,961",426,"7,120",NP,220,195,NP,San Juan del Sur


In [363]:
w1.columns

Index(['Municipios', 'Eleccion1', 'Votos depositados', 'Votos Validos', 'PLC',
       'AFSLN', 'YATAMA', 'ALN', 'APRE', 'APLI', 'Municipio'],
      dtype='object')

In [365]:
cols = ['Votos depositados', 'Votos Validos', 'PLC', 'AFSLN',
       'YATAMA', 'ALN', 'APRE', 'APLI']

w1[cols]=w1[cols].astype(int)

ValueError: invalid literal for int() with base 10: '19,911'

### I'm going to replace some characters, like dots, commas ando others, and then convert the columns to integer

In [371]:
w1[cols] = w1[cols].replace('\.','',regex=True).replace(',','',regex=True)\
.replace('\n¡','',regex=True).replace('NP','0',regex=True).replace(' ','',regex=True).astype(int)
w1

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio
0,BOACO,ALCALDE Y VICEALCALDE,19911,19020,1704,14810,0,872,832,802,Boaco
1,CAMOAPA,ALCALDE Y VICEALCALDE,13542,13016,3295,8098,0,583,524,516,Camoapa
2,SAN JOSE DE LOS REMATES,ALCALDE Y VICEALCALDE,4300,4080,1613,2030,0,149,132,156,San José de Los Remates
3,SAN LORENZO,ALCALDE Y VICEALCALDE,9005,8856,745,7223,0,272,312,304,San Lorenzo
4,SANTA LUCIA,ALCALDE Y VICEALCALDE,2597,2537,382,2021,0,20,97,17,Santa Lucía
...,...,...,...,...,...,...,...,...,...,...,...
301,POTOSI,CONCEJALES MUNICIPALES,6592,6349,303,5732,0,199,0,115,Potosí
302,RIVAS,CONCEJALES MUNICIPALES,19759,18861,967,16659,0,818,0,417,Rivas
303,SAN JORGE,CONCEJALES MUNICIPALES,4570,4353,350,3782,0,132,0,89,San Jorge
304,SAN JUAN DEL SUR,CONCEJALES MUNICIPALES,8301,7961,426,7120,0,220,195,0,San Juan del Sur


In [374]:
for col in w1:
    print(w1[col].unique())

['BOACO' 'CAMOAPA' 'SAN JOSE DE LOS REMATES' 'SAN LORENZO' 'SANTA LUCIA'
 'TEUSTEPE' 'DIRIAMBA' 'DOLORES' 'EL ROSARIO' 'JINOTEPE' 'LA CONQUISTA'
 'LA PAZ DE CARAZO' 'SAN MARCOS' 'SANTA TERESA' 'CIIIC1I1GALPA'
 'CHINANDEGA' 'CINCO PINOS' 'CORINTO' 'EL REALEJO' 'EL VIEJO' 'POSOLTEGA'
 'PUERTO MORAZAN' 'SAN FRANCISCO DEL \nNORTE' 'SAN PEDRO DEL NORTE'
 'SANTO TOMAS DEL NORTE' 'SOMOTILLO' 'VILLANUEVA' 'CHICIIIGALPA'
 'SANTO TOMAS DEL ÑOR TE' 'ACOYA PA' 'COMALAPA' 'EL AYOTE' 'EL CORAL'
 'EL RAMA' 'JUIGALPA' 'LA LIBERTAD' 'MUELLE DE LOS BUEYES' 'NUEVA GUINEA'
 'SAN FRANCISCO DE CUAPA' 'SAN PEDRO DE LOVAGO' 'SANTO DOMINGO'
 'SANTO TOMAS' 'VILLA SANDINO' 'ACOYAPA' 'CONDEGA' 'ESTELI' 'LA TRINIDAD'
 'PUEBLO NUEVO' 'SAN JUAN DE LIMAY' 'SAN NICOLAS' 'DIRIA' 'DIRIOMO'
 'GRANADA' 'NANDAIME' 'ELCUA' 'JINOTEGA' 'LA CONCORDIA'
 'SAN JOSE DE BOCAY' 'SAN RAFAEL DEL NORTE' 'SAN SEBASTIAN DE YALI'
 'SANTA MARIA DE \nPANTASMA' 'WIWILI-JINOTEGA' 'SAN JOSE DE BOCA Y'
 'W1WILI-JINOTEGA' 'ACHUAPA' 'EL JICARAL' 

In [375]:
w1.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Municipios         306 non-null    object
 1   Eleccion1          306 non-null    object
 2   Votos depositados  306 non-null    int64 
 3   Votos Validos      306 non-null    int64 
 4   PLC                306 non-null    int64 
 5   AFSLN              306 non-null    int64 
 6   YATAMA             306 non-null    int64 
 7   ALN                306 non-null    int64 
 8   APRE               306 non-null    int64 
 9   APLI               306 non-null    int64 
 10  Municipio          306 non-null    object
dtypes: int64(8), object(3)
memory usage: 26.4+ KB


### Now, we have a perfect dataset, which is going to be saved as a csv to make the visualization on Tableau.

In [377]:
w1.Municipio.value_counts()

Boaco                   2
San Isidro              2
Esquipulas              2
Matagalpa               2
Matiguás                2
                       ..
Jinotega                2
La Concordia            2
San José de Bocay       2
San Rafael del Norte    2
Tola                    2
Name: Municipio, Length: 153, dtype: int64

In [383]:
w1.to_csv('Nicaragua_Results_Municipality_Elections2022.csv')

In [379]:
w1[['PLC', 'AFSLN','YATAMA', 'ALN', 'APRE', 'APLI']].idxmax(axis=1).tolist()

['AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',
 'AFSLN',


In [382]:
w1['Winner'] = w1[['PLC', 'AFSLN','YATAMA', 'ALN', 'APRE', 'APLI']].idxmax(axis=1)
w1

,Municipios,Eleccion1,Votos depositados,Votos Validos,PLC,AFSLN,YATAMA,ALN,APRE,APLI,Municipio,Winner
0,BOACO,ALCALDE Y VICEALCALDE,19911,19020,1704,14810,0,872,832,802,Boaco,AFSLN
1,CAMOAPA,ALCALDE Y VICEALCALDE,13542,13016,3295,8098,0,583,524,516,Camoapa,AFSLN
2,SAN JOSE DE LOS REMATES,ALCALDE Y VICEALCALDE,4300,4080,1613,2030,0,149,132,156,San José de Los Remates,AFSLN
3,SAN LORENZO,ALCALDE Y VICEALCALDE,9005,8856,745,7223,0,272,312,304,San Lorenzo,AFSLN
4,SANTA LUCIA,ALCALDE Y VICEALCALDE,2597,2537,382,2021,0,20,97,17,Santa Lucía,AFSLN
...,...,...,...,...,...,...,...,...,...,...,...,...
301,POTOSI,CONCEJALES MUNICIPALES,6592,6349,303,5732,0,199,0,115,Potosí,AFSLN
302,RIVAS,CONCEJALES MUNICIPALES,19759,18861,967,16659,0,818,0,417,Rivas,AFSLN
303,SAN JORGE,CONCEJALES MUNICIPALES,4570,4353,350,3782,0,132,0,89,San Jorge,AFSLN
304,SAN JUAN DEL SUR,CONCEJALES MUNICIPALES,8301,7961,426,7120,0,220,195,0,San Juan del Sur,AFSLN
